#### Imports

In [ ]:
from typing_extensions import NotRequired, TypedDict
from functools import reduce

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import xarray as xr
import textwrap


from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    label_from_attrs,
    set_custom_rcParams,
    add_subplotlabel,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

from sdm_eurec4a import RepositoryPath
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def set_xticks_time(ax):
    xticks = [0, 500, 1000]
    ax.set_xticks(xticks)


def set_yticks_height(ax):
    yticks = [0, 500, 1000, 1500, 2000]
    ax.set_yticks(yticks)


def set_yticks_height_km(ax):
    yticks = [0, 0.5, 1, 1.5, 2]
    ax.set_yticks(yticks)


def set_logxticks_meter(ax):
    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logxticks_micrometer(ax):
    xticks = [1e-3, 1e0, 1e3]
    xticklabels = [r"$10^{-3}$", r"$10^{0}$", r"$10^{3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logtyticks_psd(ax):
    yticks = [1e0, 1e6]
    yticklabels = [r"$10^0$", r"$10^6$"]
    ax.set_yticks(yticks, yticklabels)


def set_yticks_lwc(ax):
    ax.set_yticks([0, 0.1, 0.2])

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
plt.rcParams.update(
    {
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.spines.left": False,
        "axes.spines.bottom": False,
    }
)

dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

subdata_dir = "output_v3.5"
data_path = Path("/home/m/m301096/CLEO/data/") / subdata_dir

ds_subpath = "combined/eulerian_dataset_combined_v2.nc"

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results/")
print(script_dir)


fig_dir = repo_path / "results" / script_dir.relative_to(repo_path) / subdata_dir / "microphysics"
print(fig_dir)

fig_dir.mkdir(parents=True, exist_ok=True)


(fig_dir / "profiles").mkdir(parents=False, exist_ok=True)
(fig_dir / "comparison").mkdir(parents=False, exist_ok=True)
(fig_dir / "temporal_evolution").mkdir(parents=False, exist_ok=True)


microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_large",
)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/thesis/results/output_v3.5/microphysics


# Set time slice to use for temporal mean and median

In [ ]:
time_slice = slice(1500, 3590)  # seconds
radius_split = 45  # µm
radius_slice = slice(1e0, None)  # µm

# Load datasets

In [ ]:
class MicrophysicDict(TypedDict):
    dataset_selected: xr.Dataset
    dataset: xr.Dataset
    microphysics: str
    microphysics_abbrev: str
    path: Path
    # linestyle: Union[str, tuple]
    color: str


class OptionalDictOfMicrophysicDict(TypedDict):
    null_microphysics: NotRequired[MicrophysicDict]
    condensation: NotRequired[MicrophysicDict]
    collision_condensation: NotRequired[MicrophysicDict]
    coalbure_condensation_large: NotRequired[MicrophysicDict]
    coalbure_condensation_small: NotRequired[MicrophysicDict]


class DictOfMicrophysicDict(TypedDict):
    null_microphysics: MicrophysicDict
    condensation: MicrophysicDict
    collision_condensation: MicrophysicDict
    coalbure_condensation_cke: MicrophysicDict
    coalbure_condensation_large: MicrophysicDict
    coalbure_condensation_small: MicrophysicDict


data_dict = DictOfMicrophysicDict(
    null_microphysics=MicrophysicDict(
        microphysics="Null microphysics",
        microphysics_abbrev="$NMP$",
        path=Path(),
        dataset=xr.Dataset(),
        dataset_selected=xr.Dataset(),
        color="k",
    ),
    condensation=MicrophysicDict(
        microphysics="Condensation/Evaporation",
        microphysics_abbrev="$CE$",
        path=Path(),
        dataset=xr.Dataset(),
        dataset_selected=xr.Dataset(),
        color="k",
    ),
    collision_condensation=MicrophysicDict(
        microphysics="Coll-coal, cond./evap.",
        microphysics_abbrev="$COAL$",
        path=Path(),
        dataset=xr.Dataset(),
        dataset_selected=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_cke=MicrophysicDict(
        microphysics="Coll-coal-breakup n by CKE and cond./evap.",
        microphysics_abbrev="$CBR_{CKE}$",
        path=Path(),
        dataset=xr.Dataset(),
        dataset_selected=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_large=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=125) and cond./evap.",
        microphysics_abbrev="$CBR_{M}$",
        path=Path(),
        dataset=xr.Dataset(),
        dataset_selected=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_small=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=5) and cond./evap.",
        microphysics_abbrev="$CBR_{F}$",
        path=Path(),
        dataset=xr.Dataset(),
        dataset_selected=xr.Dataset(),
        color="k",
    ),
)

colors_dict = dict(
    null_microphysics="grey",
    condensation=default_colors[2],
    collision_condensation=default_colors[1],
    coalbure_condensation_cke="k",
    coalbure_condensation_large=default_colors[3],
    coalbure_condensation_small=default_colors[4],
)

for mp in data_dict:
    data_dict[mp]["path"] = data_path / f"{mp}" / ds_subpath
    data_dict[mp]["color"] = colors_dict[mp]

for key in data_dict:
    ds = xr.open_dataset(data_dict[key]["path"], chunks={"cloud_id": 2})
    # ds = ds.sel(cloud_id = [18, 301])
    ds.attrs.update(microphysics=data_dict[key]["microphysics"])
    ds.attrs.update(microphysics_short=key)
    ds.attrs.update(microphysics_abbrev=data_dict[key]["microphysics_abbrev"])

    data_dict[key]["dataset"] = ds

# ---------------------------------------------------- #
# Reindex the datasets to have the same radius bins
# ---------------------------------------------------- #

combined_radius_bins = reduce(
    np.union1d, [data_dict[mp]["dataset"]["radius_bins"].values for mp in data_dict]
)
fill_value = np.nan
print("Number of radius bins:", len(combined_radius_bins))
print("Fill value:", fill_value)
for mp in data_dict:
    data_dict[mp]["dataset"] = data_dict[mp]["dataset"].reindex(
        radius_bins=combined_radius_bins, fill_value=fill_value
    )

Number of radius bins: 76
Fill value: nan


### All clouds which are simulated

In [ ]:
intersect_cloud_ids = reduce(
    np.intersect1d, [data_dict[key]["dataset"]["cloud_id"].data for key in data_dict]
)
for mp in data_dict:
    print(mp, len(data_dict[mp]["dataset"]["cloud_id"]))

print("\nIntersect:", len(intersect_cloud_ids))
print("cloud_ids:", intersect_cloud_ids)

null_microphysics 94
condensation 78
collision_condensation 83
coalbure_condensation_cke 91
coalbure_condensation_large 93
coalbure_condensation_small 94

Intersect: 66
cloud_ids: [  9  11  18  20  21  22  65  67  68  71  72  73  74  88  94 110 113 114
 130 135 136 142 194 197 198 199 201 203 205 207 208 211 212 213 214 215
 217 218 219 220 221 222 223 224 230 233 235 236 237 292 293 295 296 301
 303 305 306 307 308 309 311 312 314 359 361 362]


In [ ]:
clouds_dict = {
    "222": dict(
        cloud_id=222,
        color="r",
    ),
    "142": dict(
        cloud_id=142,
        color="b",
    ),
}
for key in data_dict:
    ds = data_dict[key]["dataset"]
    for cloud_id in clouds_dict:
        cloud_id = clouds_dict[cloud_id]["cloud_id"]
        is_in = cloud_id in ds["cloud_id"]
        print(f"{cloud_id}, {key}, {is_in}")

222, null_microphysics, True
142, null_microphysics, True
222, condensation, True
142, condensation, True
222, collision_condensation, True
142, collision_condensation, True
222, coalbure_condensation_cke, True
142, coalbure_condensation_cke, True
222, coalbure_condensation_large, True
142, coalbure_condensation_large, True
222, coalbure_condensation_small, True
142, coalbure_condensation_small, True


### Add further variables e.g. latent cooling

In [ ]:
# NOTE: For now, it needs to be divided by 2 s to get values per second Bue to a bug.
def add_variables(
    ds: xr.Dataset, latent_heat_of_condensation: float = 2.265e6, time_slice=time_slice  # J kg-1
):
    # fix attributes from basic variables
    ds["mass_represented"].attrs["long_name"] = "Mass"

    ds["radius_bins"].attrs["long_name"] = "Radius"
    ds["radius_bins"].attrs["description"] = "Bin centers of the radius bins"
    ds["radius_bins"].attrs["units"] = "$\\mu m$"

    ds["relative_humidity"].attrs["long_name"] = "Relative humidity"
    ds["relative_humidity"].attrs["units"] = "$\\%$"

    ds["gridbox_thickness"] = ds["gridbox_top"] - ds["gridbox_bottom"]
    ds["gridbox_thickness"].attrs["long_name"] = "Gridbox thickness"
    ds["gridbox_thickness"].attrs["units"] = "$m$"

    ds["cloud_base_height"] = ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"])
    ds["cloud_base_height"].attrs["long_name"] = "Cloud base height"
    ds["cloud_base_height"].attrs["units"] = "$m$"

    # It seems that xi was stored as an integer. This is not wanted, because nan values will just be large integers.
    ds["xi"] = ds["xi"].astype(float)
    ds["xi"] = ds["xi"].where(ds["xi"] < 1e12)
    ds["xi"].attrs["units"] = ""
    ds["xi"].attrs["long_name"] = "Real droplet num. conc."
    ds["xi"].attrs["description"] = "Real droplet number concentration in the gridbox"

    ds["xi_per_volume"] = ds["xi"] / ds["gridbox_volume"]
    ds["xi_per_volume"].attrs["long_name"] = ds["xi"].attrs["long_name"]
    ds["xi_per_volume"].attrs["units"] = "$m^{-3}$"
    ds["xi_per_volume"].attrs[
        "description"
    ] = "Real droplet number concentration in one cubic meter of air"

    ds["number_superdroplets_per_volume"] = 1000 * ds["number_superdroplets"] / ds["gridbox_volume"]
    ds["number_superdroplets_per_volume"].attrs["units"] = "$10^3m^{-3}$"
    ds["number_superdroplets_per_volume"].attrs[
        "long_name"
    ] = "Number of superdroplets in 1000 cubic meter of air"

    ds["mass_represented_per_volume"] = 1e3 * ds["mass_represented"] / ds["gridbox_volume"]
    ds["mass_represented_per_volume"].attrs["units"] = "$g m^{-3}$"
    ds["mass_represented_per_volume"].attrs["long_name"] = "Mass"
    ds["mass_represented_per_volume"].attrs[
        "description"
    ] = "Mass of the represented droplets in one cubic meter of air"

    ds["massdelta_condensation_per_volume"] = ds["massdelta_condensation"] / ds["gridbox_volume"]
    ds["massdelta_condensation_per_volume"].attrs["units"] = "$kg m^{-3} s^{-1}$"
    ds["massdelta_condensation_per_volume"].attrs["long_name"] = "Condensation rate"
    ds["massdelta_condensation_per_volume"].attrs[
        "description"
    ] = "Condensation rate in one cubic meter of air"

    ds["latent_cooling_full"] = (
        -1e3
        * ds["massdelta_condensation"].sel(time=time_slice).where(ds["sub_cloud_layer_mask"])
        * latent_heat_of_condensation
    )  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_cooling_full"].attrs["units"] = "$mW m^{-3}$"
    ds["latent_cooling_full"].attrs["long_name"] = "Latent cooling"
    ds["latent_cooling_full"].attrs["description"] = "Latent cooling for all timesteps"

    ds["latent_cooling_full_gi"] = 1e-3 * ds["latent_cooling_full"] * ds["gridbox_thickness"]
    ds["latent_cooling_full_gi"].attrs["units"] = "$W m^{-2}$"
    ds["latent_cooling_full_gi"].attrs["long_name"] = "Latent cooling"
    ds["latent_cooling_full_gi"].attrs[
        "description"
    ] = "Gridbox integrated latent cooling for all timesteps"

    ds["latent_cooling_full_ci"] = ds["latent_cooling_full_gi"].sum("gridbox", keep_attrs=True)
    ds["latent_cooling_full_ci"].attrs["units"] = "$W m^{-2}$"
    ds["latent_cooling_full_ci"].attrs["long_name"] = "Column int. latent cooling"
    ds["latent_cooling_full_ci"].attrs[
        "description"
    ] = "Column integrated latent cooling for all timesteps"

    ds["latent_cooling"] = (
        -1e3
        * ds["massdelta_condensation"]
        .sel(time=time_slice)
        .mean("time", keep_attrs=True, skipna=True)
        .where(ds["sub_cloud_layer_mask"])
        * latent_heat_of_condensation
    )
    ds["latent_cooling"].attrs["units"] = "$mW m^{-3}$"  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_cooling"].attrs["long_name"] = "Latent cooling"
    ds["latent_cooling"].attrs[
        "description"
    ] = "Latent cooling for the sub cloud layer. Calculated with the condensation monitor of CLEO."

    ds["latent_cooling_gi"] = 1e-3 * ds["latent_cooling"] * ds["gridbox_thickness"]
    ds["latent_cooling_gi"].attrs["units"] = "$W m^{-2}$"
    ds["latent_cooling_gi"].attrs["long_name"] = "Latent cooling"
    ds["latent_cooling_gi"].attrs[
        "description"
    ] = "Gridbox integrated latent cooling for the sub cloud layer. Calculated with the condensation monitor of CLEO."

    ds["latent_cooling_ci"] = ds["latent_cooling_gi"].sum("gridbox", keep_attrs=True)
    ds["latent_cooling_ci"].attrs["units"] = "$W m^{-2}$"
    ds["latent_cooling_ci"].attrs["long_name"] = "Column int. latent cooling"
    ds["latent_cooling_ci"].attrs[
        "description"
    ] = "Column integrated latent cooling for the sub cloud layer. Calculated with the condensation monitor of CLEO."

    ds["latent_cooling_radius_bins"] = (
        -1e3
        * ds["mass_difference_per_volume"].sel(time=time_slice).where(ds["sub_cloud_layer_mask"])
        * latent_heat_of_condensation
    )
    ds["latent_cooling_radius_bins"].attrs["units"] = "$mW m^{-3}$"  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_cooling_radius_bins"].attrs["long_name"] = "Latent cooling"
    ds["latent_cooling_radius_bins"].attrs[
        "description"
    ] = "Latent cooling for the sub cloud layer. Calculated with the mass difference from lagrangian particles."

# Analysis of the domain and metrics

## Identify outliers


It seems the outlier is cloud ``296``

In [ ]:
# remove outlier cloud_id 296
cloud_id_selection = intersect_cloud_ids[intersect_cloud_ids != 296]

for key in data_dict:
    data_dict[key]["dataset_selected"] = data_dict[key]["dataset"].sel(cloud_id=cloud_id_selection)

for mp in data_dict:
    ds = data_dict[mp]["dataset_selected"]
    add_variables(ds)


null_microphysics = data_dict["null_microphysics"]["dataset_selected"]
condensation = data_dict["condensation"]["dataset_selected"]
collision_condensation = data_dict["collision_condensation"]["dataset_selected"]
coalbure_condensation_cke = data_dict["coalbure_condensation_cke"]["dataset_selected"]
coalbure_condensation_large = data_dict["coalbure_condensation_large"]["dataset_selected"]
coalbure_condensation_small = data_dict["coalbure_condensation_small"]["dataset_selected"]

# Vertical profiles analysis

### Data of all microphysics

In [ ]:
SETUP_KEYS = [
    # "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

In [ ]:
latent_cooling_profiles = dict()

for key in SETUP_KEYS:
    print(key)
    ds = data_dict[key]["dataset_selected"]

    data = ds["latent_cooling_full"].sel(time=time_slice).where(ds["sub_cloud_layer_mask"]).compute()
    attrs = data.attrs

    individual_mean, individual_std = mean_and_stderror_of_mean(
        data=data,
        dims=("time",),
    )
    individual_mean = individual_mean.compute()
    individual_mean.attrs.update(**attrs)
    individual_std = individual_std.compute()

    total_mean, total_std = mean_and_stderror_of_mean(
        data=individual_mean,
        dims=("cloud_id",),
        data_std=individual_std,
    )
    total_mean = total_mean.compute()
    total_mean.attrs.update(**attrs)
    total_std = total_std.compute()

    latent_cooling_profiles[key] = dict(
        individual_mean=individual_mean,
        individual_std=individual_std,
        total_mean=total_mean,
        total_std=total_std,
    )


diff_latent_cooling_profiles = dict()
for key in SETUP_KEYS:
    print(key)
    attrs = latent_cooling_profiles[key]["individual_mean"].attrs
    individual_mean = (
        latent_cooling_profiles[key]["individual_mean"]
        - latent_cooling_profiles["condensation"]["individual_mean"]
    )
    individual_std: xr.DataArray = np.sqrt(
        latent_cooling_profiles[key]["individual_std"] ** 2
        + latent_cooling_profiles["condensation"]["individual_std"] ** 2
    )

    individual_mean = individual_mean.compute()
    individual_mean.attrs.update(**attrs)
    individual_std = individual_std.compute()

    total_mean, total_std = mean_and_stderror_of_mean(
        data=individual_mean,
        dims=("cloud_id",),
        data_std=individual_std,
    )
    total_mean = total_mean.compute()
    total_mean.attrs.update(**attrs)
    total_std = total_std.compute()

    diff_latent_cooling_profiles[key] = dict(
        individual_mean=individual_mean,
        individual_std=individual_std,
        total_mean=total_mean,
        total_std=total_std,
    )


latent_cooling_profiles_33 = dict()

for key in data_dict:
    print(key)
    ds = data_dict[key]["dataset_selected"]

    data = (
        ds["latent_cooling_full"]
        .sel(time=time_slice, cloud_id=(ds["max_gridbox"] == 33))
        .where(ds["sub_cloud_layer_mask"])
        .compute()
    )
    attrs = data.attrs

    individual_mean, individual_std = mean_and_stderror_of_mean(
        data=data,
        dims=("time",),
    )
    individual_mean = individual_mean.compute()
    individual_mean.attrs.update(**attrs)
    individual_std = individual_std.compute()

    total_mean, total_std = mean_and_stderror_of_mean(
        data=individual_mean,
        dims=("cloud_id",),
        data_std=individual_std,
    )
    total_mean = total_mean.compute()
    total_mean.attrs.update(**attrs)
    total_std = total_std.compute()

    latent_cooling_profiles_33[key] = dict(
        individual_mean=individual_mean,
        individual_std=individual_std,
        total_mean=total_mean,
        total_std=total_std,
    )

diff_latent_cooling_profiles_33 = dict()
for key in SETUP_KEYS:
    print(key)
    attrs = latent_cooling_profiles_33[key]["individual_mean"].attrs
    individual_mean = (
        latent_cooling_profiles_33[key]["individual_mean"]
        - latent_cooling_profiles_33["condensation"]["individual_mean"]
    )
    individual_std: xr.DataArray = np.sqrt(
        latent_cooling_profiles_33[key]["individual_std"] ** 2
        + latent_cooling_profiles_33["condensation"]["individual_std"] ** 2
    )

    individual_mean = individual_mean.compute()
    individual_mean.attrs.update(**attrs)
    individual_std = individual_std.compute()

    total_mean, total_std = mean_and_stderror_of_mean(
        data=individual_mean,
        dims=("cloud_id",),
        data_std=individual_std,
    )
    total_mean = total_mean.compute()
    total_mean.attrs.update(**attrs)
    total_std = total_std.compute()

    diff_latent_cooling_profiles_33[key] = dict(
        individual_mean=individual_mean,
        individual_std=individual_std,
        total_mean=total_mean,
        total_std=total_std,
    )

condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small


condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small
null_microphysics
condensation
collision_condensation
coalbure_condensation_cke
coalbure_condensation_large
coalbure_condensation_small
condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small


In [ ]:
var = "latent_cooling"

fig = plt.figure(figsize=(9, 5))
subfigs = fig.subfigures(nrows=1, ncols=2, wspace=0.01)

fig_left = subfigs[0]
fig_right = subfigs[1]

ax_left = fig_left.subplots(nrows=1, ncols=1)
axs_right = fig_right.subplots(nrows=2, ncols=2, sharey=True, sharex=True)
axs_right_flat = axs_right.flatten()
axs_all = np.concatenate([[ax_left], axs_right_flat])

for idx, mp in enumerate(SETUP_KEYS):
    ds = data_dict[mp]["dataset_selected"]
    y = ds["gridbox_coord3"].min("cloud_id")
    title = ds.attrs["microphysics_abbrev"]

    axs_right_flat[idx].set_title(title)

    color = data_dict[mp]["color"]
    m_color = adjust_lightness_array([color], 0.9)[0]

    latent_cooling_dict = latent_cooling_profiles[mp]

    # plot for individual clouds with mean and std in right axes objects
    m = latent_cooling_dict["individual_mean"]
    s = latent_cooling_dict["individual_std"]

    axs_right_flat[idx].plot(
        m.T,
        y,
        color=color,
        linestyle="-",
        alpha=0.2,
        zorder=2,
    )
    # for cloud_id in latent_cooling_dict["individual_mean"]["cloud_id"]:
    #     mm = m.sel(cloud_id=cloud_id)
    #     ss = s.sel(cloud_id=cloud_id)
    #     axs_right_flat[idx].fill_betweenx(
    #         y,
    #         mm + 2 * ss,
    #         mm - 2 * ss,
    #         color=color,
    #         edgecolor="None",
    #         alpha=0.1,
    #         zorder=1,
    #     )

    # plot the overall mean and std in the left axes object
    for ax in [ax_left, axs_right_flat[idx]]:
        m = latent_cooling_dict["total_mean"]
        s = latent_cooling_dict["total_std"]

        ax.fill_betweenx(
            y,
            m - 2 * s,
            m + 2 * s,
            color=color,
            alpha=0.1,
            edgecolor="None",
            zorder=4,
        )
        ax.plot(
            m,
            y,
            color=m_color,
            linestyle="-",
            linewidth=2,
            alpha=0.8,
            zorder=5,
            label=ds.attrs["microphysics_abbrev"],
        )


ax_left.legend(loc="upper right")

# ax_left.axvline(0, color="black", linestyle="--", alpha=0.5)
ax_left.set_ylabel("Height [m]")

subfigs[0].supxlabel(label_from_attrs(m))
subfigs[1].supxlabel(label_from_attrs(m))

# for ax in axs_right_flat:
# ax.set_yticklabels([])

for ax in axs_all:
    ax.set_ylim(0, 1000)
    ax.set_xlim(0, 250)

add_subplotlabel(axs_all, location="title")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-latent_cooling-all.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-latent_cooling-all.pdf")

ax_left.set_xlim(28, 62)
ax_left.set_xticks([30, 40, 50, 60])

for ax in axs_right_flat:
    ax.set_xticks([25, 50, 75])
    ax.set_xlim(20, 80)


fig.tight_layout()
fig.savefig(
    bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-latent_cooling-all-zoomed.svg"
)
fig.savefig(
    bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-latent_cooling-all-zoomed.pdf"
)

In [ ]:
var = "latent_cooling"

fig = plt.figure(figsize=(9, 5))
subfigs = fig.subfigures(nrows=1, ncols=2, wspace=0.01)

fig_left = subfigs[0]
fig_right = subfigs[1]

ax_left = fig_left.subplots(nrows=1, ncols=1)
axs_right = fig_right.subplots(nrows=2, ncols=2, sharey=True, sharex=True)
axs_right_flat = axs_right.flatten()
axs_all = np.concatenate([[ax_left], axs_right_flat])

for idx, mp in enumerate(SETUP_KEYS):
    ds = data_dict[mp]["dataset_selected"]
    y = ds["gridbox_coord3"].min("cloud_id")
    title = ds.attrs["microphysics_abbrev"]

    axs_right_flat[idx].set_title(title)

    color = data_dict[mp]["color"]
    m_color = adjust_lightness_array([color], 0.9)[0]

    latent_cooling_dict = latent_cooling_profiles_33[mp]

    # plot for individual clouds with mean and std in right axes objects
    m = latent_cooling_dict["individual_mean"]
    s = latent_cooling_dict["individual_std"]

    axs_right_flat[idx].plot(
        m.T,
        y,
        color=color,
        linestyle="-",
        alpha=0.2,
        zorder=2,
    )
    # for cloud_id in latent_cooling_dict["individual_mean"]["cloud_id"]:
    #     mm = m.sel(cloud_id=cloud_id)
    #     ss = s.sel(cloud_id=cloud_id)
    #     axs_right_flat[idx].fill_betweenx(
    #         y,
    #         mm + 2 * ss,
    #         mm - 2 * ss,
    #         color=color,
    #         edgecolor="None",
    #         alpha=0.1,
    #         zorder=1,
    #     )

    # plot the overall mean and std in the left axes object
    for ax in [ax_left, axs_right_flat[idx]]:
        m = latent_cooling_dict["total_mean"]
        s = latent_cooling_dict["total_std"]

        ax.fill_betweenx(
            y,
            m - 2 * s,
            m + 2 * s,
            color=color,
            alpha=0.1,
            edgecolor="None",
            zorder=4,
        )
        ax.plot(
            m,
            y,
            color=m_color,
            linestyle="-",
            linewidth=2,
            alpha=0.8,
            zorder=5,
            label=ds.attrs["microphysics_abbrev"],
        )


ax_left.legend(loc="upper right")

ax_left.axvline(0, color="black", linestyle="--", alpha=0.5)
ax_left.set_ylabel("Height [m]")

subfigs[0].supxlabel(label_from_attrs(m))
subfigs[1].supxlabel(label_from_attrs(m))

for ax in axs_right_flat:
    ax.set_yticklabels([])

for ax in axs_all:
    ax.set_ylim(0, 650)
    ax.set_xlim(0, 250)

add_subplotlabel(axs_all, location="title")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-latent_cooling-33.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-latent_cooling-33.pdf")

ax_left.set_xlim(38, 62)
ax_left.set_xticks([40, 50, 60])

for ax in axs_right_flat:
    ax.set_xticks([25, 50, 75])
    ax.set_xlim(20, 80)

fig.tight_layout()

fig.savefig(
    bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-latent_cooling-33-zoomed.svg"
)
fig.savefig(
    bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-latent_cooling-33-zoomed.pdf"
)

### Difference between ``Condensation`` and other microphysics

In [ ]:
var = "latent_cooling"

fig = plt.figure(figsize=(9, 5))
subfigs = fig.subfigures(nrows=1, ncols=2, wspace=0.01)

fig_left = subfigs[0]
fig_right = subfigs[1]

ax_left = fig_left.subplots(nrows=1, ncols=1)
axs_right = fig_right.subplots(nrows=2, ncols=2, sharey=True, sharex=True)
axs_right_flat = axs_right.flatten()
axs_all = np.concatenate([[ax_left], axs_right_flat])


# plot the condensation microphysics
color = data_dict["condensation"]["color"]
m_color = adjust_lightness_array([color], 0.9)[0]

# m = diff_latent_cooling_profiles["condensation"]["total_mean"]
# s = diff_latent_cooling_profiles["condensation"]["total_std"]

# ax_left.fill_betweenx(
#     y,
#     m - 2 * s,
#     m + 2 * s,
#     color=color,
#     alpha=0.1,
#     edgecolor="None",
#     zorder=4,
# )
# ax_left.plot(
#     m,
#     y,
#     color=m_color,
#     linestyle="-",
#     linewidth=2,
#     alpha=0.8,
#     zorder=5,
#     label=data_dict["condensation"]["microphysics_abbrev"],
# )


for idx, mp in enumerate(SETUP_KEYS):
    ds = data_dict[mp]["dataset_selected"]
    y = ds["gridbox_coord3"].min("cloud_id")
    title = ds.attrs["microphysics_abbrev"]

    axs_right_flat[idx].set_title(title)

    color = data_dict[mp]["color"]
    m_color = adjust_lightness_array([color], 0.9)[0]

    latent_cooling_dict = diff_latent_cooling_profiles[mp]

    # plot for individual clouds with mean and std in right axes objects
    m = latent_cooling_dict["individual_mean"]
    s = latent_cooling_dict["individual_std"]

    axs_right_flat[idx].plot(
        m.T,
        y,
        color=color,
        linestyle="-",
        alpha=0.2,
        zorder=2,
    )

    for cloud_id in latent_cooling_dict["individual_mean"]["cloud_id"]:
        mm = m.sel(cloud_id=cloud_id)
        ss = s.sel(cloud_id=cloud_id)
        axs_right_flat[idx].fill_betweenx(
            y,
            mm + 2 * ss,
            mm - 2 * ss,
            color=color,
            edgecolor="None",
            alpha=0.1,
            zorder=1,
        )

    # plot the overall mean and std in the left axes object
    for ax in [ax_left, axs_right_flat[idx]]:
        m = latent_cooling_dict["total_mean"]
        s = latent_cooling_dict["total_std"]

        ax.fill_betweenx(
            y,
            m - 2 * s,
            m + 2 * s,
            color=color,
            alpha=0.1,
            edgecolor="None",
            zorder=4,
        )
        ax.plot(
            m,
            y,
            color=m_color,
            linestyle="-",
            linewidth=2,
            alpha=0.8,
            zorder=5,
            label=ds.attrs["microphysics_abbrev"],
        )


ax_left.legend(loc="upper left")
ax_left.set_ylabel("Height [m]")

for ax in axs_all:
    ax.axvline(0, color="black", linestyle="--", alpha=0.5)

subfigs[0].supxlabel(label_from_attrs(m))
subfigs[1].supxlabel(label_from_attrs(m))

# for ax in axs_right_flat:
#     ax.set_yticklabels([])

for ax in axs_all:
    ax.set_ylim(0, 700)
    ax.set_xlim(-5, 75)

add_subplotlabel(axs_all, location="title")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-diff-latent_cooling-all.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-diff-latent_cooling-all.pdf")

ax_left.set_xlim(-5, 1)
ax_left.set_xticks([-4, -2, 0])

for ax in axs_right_flat:
    ax.set_xticks([-8, -4, 0, 4])
    ax.set_xlim(-8, 4)

fig.tight_layout()

fig.savefig(
    bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-diff-latent_cooling-all-zoom.svg"
)
fig.savefig(
    bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-diff-latent_cooling-all-zoom.pdf"
)

In [ ]:
var = "latent_cooling"

fig = plt.figure(figsize=(9 * 3 / 4, 5))
subfigs = fig.subfigures(nrows=1, ncols=2, wspace=0.01, width_ratios=[2, 1])

fig_left = subfigs[0]
fig_right = subfigs[1]

ax_left = fig_left.subplots(nrows=1, ncols=1)
axs_right = fig_right.subplots(nrows=2, ncols=1, sharey=True, sharex=True)
axs_right_flat = axs_right.flatten()
axs_all = np.concatenate([[ax_left], axs_right_flat])


# plot the condensation microphysics
color = data_dict["condensation"]["color"]
m_color = adjust_lightness_array([color], 0.9)[0]

m = diff_latent_cooling_profiles_33["condensation"]["total_mean"]
s = diff_latent_cooling_profiles_33["condensation"]["total_std"]

ax_left.fill_betweenx(
    y,
    m - 2 * s,
    m + 2 * s,
    color=color,
    alpha=0.1,
    edgecolor="None",
    zorder=4,
)
ax_left.plot(
    m,
    y,
    color=m_color,
    linestyle="-",
    linewidth=2,
    alpha=0.8,
    zorder=5,
    label=data_dict["condensation"]["microphysics_abbrev"],
)


for idx, mp in enumerate(["collision_condensation", "coalbure_condensation_small"]):
    ds = data_dict[mp]["dataset_selected"]
    y = ds["gridbox_coord3"].min("cloud_id")
    title = ds.attrs["microphysics_abbrev"]

    axs_right_flat[idx].set_title(title)

    color = data_dict[mp]["color"]
    m_color = adjust_lightness_array([color], 0.9)[0]

    latent_cooling_dict = diff_latent_cooling_profiles_33[mp]

    # plot for individual clouds with mean and std in right axes objects
    m = latent_cooling_dict["individual_mean"]
    s = latent_cooling_dict["individual_std"]

    axs_right_flat[idx].plot(
        m.T,
        y,
        color=color,
        linestyle="-",
        alpha=0.2,
        zorder=2,
    )
    for cloud_id in latent_cooling_dict["individual_mean"]["cloud_id"]:
        mm = m.sel(cloud_id=cloud_id)
        ss = s.sel(cloud_id=cloud_id)
        axs_right_flat[idx].fill_betweenx(
            y,
            mm + 2 * ss,
            mm - 2 * ss,
            color=color,
            edgecolor="None",
            alpha=0.1,
            zorder=1,
        )

    # plot the overall mean and std in the left axes object
    for ax in [ax_left, axs_right_flat[idx]]:
        m = latent_cooling_dict["total_mean"]
        s = latent_cooling_dict["total_std"]

        ax.fill_betweenx(
            y,
            m - 2 * s,
            m + 2 * s,
            color=color,
            alpha=0.1,
            edgecolor="None",
            zorder=4,
        )
        ax.plot(
            m,
            y,
            color=m_color,
            linestyle="-",
            linewidth=2,
            alpha=0.8,
            zorder=5,
            label=ds.attrs["microphysics_abbrev"],
        )


ax_left.legend(loc="upper left")
ax_left.set_ylabel("Height [m]")

for ax in axs_all:
    ax.axvline(0, color="black", linestyle="--", alpha=0.5)

subfigs[0].supxlabel(label_from_attrs(m))
subfigs[1].supxlabel(label_from_attrs(m))

# for ax in axs_right_flat:
#     ax.set_yticklabels([])

for ax in axs_all:
    ax.set_ylim(0, 700)
    ax.set_xlim(-5, 75)


add_subplotlabel(axs_all, location="title")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-diff-latent_cooling-33.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-diff-latent_cooling-33.pdf")

ax_left.set_xlim(-5, 1)
ax_left.set_xticks([-4, -2, 0])

for ax in axs_right_flat:
    ax.set_xticks([-8, -4, 0, 4])
    ax.set_xlim(-8, 4)

fig.tight_layout()

fig.savefig(
    bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-diff-latent_cooling-33-zoom.svg"
)
fig.savefig(
    bbox_inches="tight", fname=fig_dir / "profiles" / "microphysics-diff-latent_cooling-33-zoom.pdf"
)

# Comparison at selected heights

In [ ]:
selected_gridboxes = (
    condensation["max_gridbox"].compute() - 2,
    # 15,
    1,
)

labels = (
    "Cloud base",
    "Surface",
    # f"{condensation['gridbox_coord3'].sel(gridbox = 15).isel(cloud_id= 0).values} m",
    # f"{condensation['gridbox_coord3'].sel(gridbox = 5).isel(cloud_id= 0).values} m",
    # f"{condensation['gridbox_coord3'].sel(gridbox = 1).isel(cloud_id= 0).values} m",
    # f"{condensation['gridbox_coord3'].sel(gridbox = 1).isel(cloud_id= 0).values} m",
)

needed_gridboxes = np.unique(np.hstack(selected_gridboxes))
needed_gridboxes

array([ 1, 23, 24, 25, 29, 31, 32, 33, 34, 35, 45, 49, 50])

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_large,
    coalbure_condensation_small,
]

keys = [data.attrs["microphysics_short"] for data in datas]


latent_heat_comparison_all = dict()
for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    da = (
        data["latent_cooling_radius_bins"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_id_selection)
        .sel(gridbox=needed_gridboxes)
        .median("time", keep_attrs=True)
        .fillna(0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs.update(**data["latent_cooling_radius_bins"].attrs)
    latent_heat_comparison_all[mp] = da

multiplicity_comparison_all = dict()
for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    da = (
        data["xi_per_volume"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_id_selection)
        .sel(gridbox=needed_gridboxes)
        .mean("time", keep_attrs=True)
        .fillna(0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = data["xi_per_volume"].attrs["units"]
    da.attrs["long_name"] = data["xi_per_volume"].attrs["long_name"]

    multiplicity_comparison_all[mp] = da

number_superdroplets_comparison_all = dict()
for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    da = (
        8
        * data["number_superdroplets_per_volume"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_id_selection)
        .sel(gridbox=needed_gridboxes)
        .mean("time", keep_attrs=True)
        .fillna(0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = "$\\left(8000 m^{3} \\right)^{-1}$"
    da.attrs["long_name"] = "Number of SDs"

    number_superdroplets_comparison_all[mp] = da

mass_comparison_all = dict()
for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    da = (
        1e3
        * data["mass_represented_per_volume"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_id_selection)
        .sel(gridbox=needed_gridboxes)
        .mean("time", keep_attrs=True)
        .fillna(0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = "m" + data["mass_represented_per_volume"].attrs["units"].replace("$", "")
    da.attrs["long_name"] = data["mass_represented_per_volume"].attrs["long_name"]

    mass_comparison_all[mp] = da

condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small
condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small
condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small
condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small


In [ ]:
def plot_mean_median_iqr(ax, data, dim, x_dim):
    x = data[x_dim]

    ax.plot(x, data.T, linestyle="-", marker="None", color=[0.75, 0.75, 0.75], alpha=0.2, zorder=1)

    ax.plot(
        x,
        data.median(dim),
        color="red",
        # linestyle="-",
        marker="+",
        linestyle="None",
        alpha=1,
        label="Median",
        zorder=6,
    )

    ax.fill_between(
        x,
        data.quantile(0.75, dim),
        data.quantile(0.25, dim),
        color="red",
        alpha=0.2,
        label="IQR",
        zorder=3,
    )

    m, s = mean_and_stderror_of_mean(data, dims=(dim,))

    ax.plot(
        x,
        m,
        color="darkorange",
        # linestyle="-",
        marker="x",
        linestyle="None",
        alpha=1,
        label="Mean",
        zorder=5,
    )

    # ax.fill_between(
    #     x,
    #     m - 2 * s,
    #     m + 2 * s,
    #     color="orange",
    #     alpha=0.2,
    #     label="2 SEM",
    #     zorder=3,
    # )

    return ax

## Condensation only

#### Latent cooling, mass and real droplet number concentration

In [ ]:
key = "condensation"

datas = [
    latent_heat_comparison_all[key],
    multiplicity_comparison_all[key],
    mass_comparison_all[key],
]

ylims = [
    (0, 12),
    (0, 50),
    (0, 50),
]


# use gridspec, to make len(keys) cols and len(selected_gridboxes) rows
# add a large axes around each row to add some ylabels
nrows = len(selected_gridboxes)
ncols = len(datas)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(9, 6), sharex=True, sharey=False)

for i, da in enumerate(datas):
    surface_gridbox = selected_gridboxes[1]
    cloud_base_gridbox = selected_gridboxes[0]

    # for cloud base
    data = da.sel(gridbox=cloud_base_gridbox).compute()
    plot_mean_median_iqr(ax=axs[0, i], data=data, dim="cloud_id", x_dim="radius_bins")

    # for surface
    data = da.sel(gridbox=cloud_base_gridbox).compute()
    axs[1, i].plot(
        data["radius_bins"],
        data.median("cloud_id"),
        color="black",
        linestyle=":",
        # marker="x", linestyle="None",
        alpha=1,
        linewidth=1,
        zorder=2,
        label="Cloud\nbase",
    )
    data = da.sel(gridbox=surface_gridbox).compute()
    plot_mean_median_iqr(ax=axs[1, i], data=data, dim="cloud_id", x_dim="radius_bins")

    axs[0, i].set_ylim(ylims[i])
    axs[1, i].set_ylim(ylims[i])
    axs[0, i].set_title(label_from_attrs(data, linebreak=True, units_appendix=" \\log(\\mu m)^{-1}"))


ax = axs.flatten()[0]
ax.set_xscale("log")
ax.set_yscale("linear")
ax.set_xlim(10, None)

axs[-1, 0].legend(
    loc="upper left",
    handlelength=1,
)

for idx, label in enumerate(labels):
    axs[idx, 0].set_ylabel(label)

add_subplotlabel(axs, location="title")

for ax in axs[-1, :]:
    ax.set_xlabel(label_from_attrs(data["radius_bins"]))

# fig.supxlabel(label_from_attrs(data["radius_bins"]))
fig.tight_layout()

fig.savefig(fig_dir / "comparison" / f"condensation-comparison-selected-gridboxes-zoom.pdf")

#### Difference between cloud base and surface

In [ ]:
key = "condensation"

datas = [
    latent_heat_comparison_all[key],
    multiplicity_comparison_all[key],
    mass_comparison_all[key],
]

ylims = [
    (0, 12),
    (0, 50),
    (0, 50),
]


# use gridspec, to make len(keys) cols and len(selected_gridboxes) rows
# add a large axes around each row to add some ylabels
nrows = len(selected_gridboxes) + 1
ncols = len(datas)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(9, 9), sharex=True, sharey=False)

for i, da in enumerate(datas):
    surface_gridbox = selected_gridboxes[1]
    cloud_base_gridbox = selected_gridboxes[0]

    # for cloud base
    data = da.sel(gridbox=cloud_base_gridbox).compute()
    plot_mean_median_iqr(ax=axs[0, i], data=data, dim="cloud_id", x_dim="radius_bins")

    # for surface
    data = da.sel(gridbox=cloud_base_gridbox).compute()
    axs[1, i].plot(
        data["radius_bins"],
        data.median("cloud_id"),
        color="black",
        linestyle=":",
        # marker="x", linestyle="None",
        alpha=1,
        linewidth=1,
        zorder=2,
        label="Cloud\nbase",
    )
    data = da.sel(gridbox=surface_gridbox).compute()
    plot_mean_median_iqr(ax=axs[1, i], data=data, dim="cloud_id", x_dim="radius_bins")

    # diff
    data_surface = da.sel(gridbox=selected_gridboxes[1]).drop("gridbox")
    data_cloud_base = da.sel(gridbox=selected_gridboxes[0]).drop("gridbox")

    diff = (data_surface - data_cloud_base).compute()

    plot_mean_median_iqr(ax=axs[2, i], data=diff, dim="cloud_id", x_dim="radius_bins")
    abs = np.max(np.abs(ylims[i])) / 2

    axs[2, i].set_ylim(-abs, abs)

    axs[0, i].set_ylim(ylims[i])
    axs[1, i].set_ylim(ylims[i])
    axs[0, i].set_title(label_from_attrs(data, linebreak=True, units_appendix=" \\log(\\mu m)^{-1}"))


ax = axs.flatten()[0]
ax.set_xscale("log")
ax.set_yscale("linear")
ax.set_xlim(10, None)
axs[-1, 0].legend(
    loc="upper left",
    handlelength=1,
)

for idx, label in enumerate(labels):
    axs[idx, 0].set_ylabel(label)
axs[2, 0].set_ylabel("Difference")

add_subplotlabel(axs, location="title")

for ax in axs[-1, :]:
    ax.set_xlabel(label_from_attrs(data["radius_bins"]))

# fig.supxlabel(label_from_attrs(data["radius_bins"]))
fig.tight_layout()

fig.savefig(fig_dir / "comparison" / f"condensation-comparison-selected-gridboxes-with_diff-zoom.pdf")

#### Evaporation fraction additional to latent cooling, mass and real droplet number concentration

In [ ]:
condensation["evaporated_fraction"]

In [ ]:
evap_fraction_cloud_base = (
    2
    * condensation["evaporated_fraction"]
    .sel(time=time_slice)
    .sel(gridbox=condensation["max_gridbox"] - 2)
    .mean("time", keep_attrs=True)
    .fillna(0)
    .compute()
)
evap_fraction_cloud_base.attrs.update(
    units="\\% (\\Delta t)^{-1} \\,",
    long_name="Evaporated fraction",
    description="Evaporated fraction of the cloud base",
)
evap_fraction_surface = (
    2
    * condensation["evaporated_fraction"]
    .sel(time=time_slice)
    .sel(gridbox=1)
    .mean("time", keep_attrs=True)
    .fillna(0)
    .compute()
)
evap_fraction_surface.attrs.update(
    units="\\% (\\Delta t)^{-1} \\,",
    long_name="Evaporated fraction",
    description="Evaporated fraction of the surface",
)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3.5))

plot_mean_median_iqr(ax=ax, data=evap_fraction_surface, dim="cloud_id", x_dim="radius_bins")

ax.set_yscale("log")
ax.yaxis.set_major_formatter(
    mticker.FuncFormatter(
        lambda y, pos: ("{{:.{:1d}f}}".format(int(np.maximum(-np.log10(y), 0)))).format(y)
    )
)
ax.set_ylim(0.03, 150)

ax.set_xscale("log")
ax.set_xlim(7, 1e3)

ax.legend(
    loc="lower left",
    handlelength=1,
)

ax.set_title(
    label_from_attrs(evap_fraction_surface, linebreak=True, units_appendix=" \\log(\\mu m)^{-1}")
)
ax.set_xlabel(label_from_attrs(evap_fraction_surface["radius_bins"]))
# ax.yaxis.tick_right()
# ax.yaxis.set_label_position("right")

# fig.supxlabel(label_from_attrs(data["radius_bins"]))
fig.tight_layout()
fig.savefig(
    bbox_inches="tight", fname=fig_dir / "comparison" / f"condensation-evap_fraction-surface.pdf"
)

In [ ]:
key = "condensation"

datas = [
    latent_heat_comparison_all[key],
    multiplicity_comparison_all[key],
    mass_comparison_all[key],
]

ylims = [
    (0, 12),
    (0, 50),
    (0, 50),
]


# use gridspec, to make len(keys) cols and len(selected_gridboxes) rows
# add a large axes around each row to add some ylabels
num_rows = len(selected_gridboxes)

fig, axs = plt.subplots(nrows=num_rows, ncols=4, figsize=(12, 6), sharex=True, sharey=False)

for i, da in enumerate(datas):
    surface_gridbox = selected_gridboxes[1]
    cloud_base_gridbox = selected_gridboxes[0]

    # for cloud base
    data = da.sel(gridbox=cloud_base_gridbox).compute()
    plot_mean_median_iqr(ax=axs[0, i], data=data, dim="cloud_id", x_dim="radius_bins")

    # for surface
    data = da.sel(gridbox=cloud_base_gridbox).compute()
    axs[1, i].plot(
        data["radius_bins"],
        data.median("cloud_id"),
        color="black",
        linestyle=":",
        # marker="x", linestyle="None",
        alpha=1,
        linewidth=1,
        zorder=2,
        label="Cloud\nbase",
    )
    data = da.sel(gridbox=surface_gridbox).compute()
    plot_mean_median_iqr(ax=axs[1, i], data=data, dim="cloud_id", x_dim="radius_bins")

    axs[0, i].set_ylim(ylims[i])
    axs[1, i].set_ylim(ylims[i])
    axs[0, i].set_title(label_from_attrs(data, linebreak=True, units_appendix=" \\log(\\mu m)^{-1}"))

plot_mean_median_iqr(
    ax=axs[0, i + 1], data=evap_fraction_cloud_base, dim="cloud_id", x_dim="radius_bins"
)
plot_mean_median_iqr(ax=axs[1, i + 1], data=evap_fraction_surface, dim="cloud_id", x_dim="radius_bins")
axs[0, i + 1].set_title(
    label_from_attrs(evap_fraction_surface, linebreak=True, units_appendix=" \\log(\\mu m)^{-1}")
)
for ax in axs[:, i + 1]:
    ax.set_yscale("log")
    ax.set_ylim(1e-1, 101)

for idx, label in enumerate(labels):
    axs[idx, 0].set_ylabel(label)

ax = axs.flatten()[0]
ax.set_xscale("log")
ax.set_yscale("linear")
ax.set_xlim(10, None)
axs[-1, 0].legend(
    loc="upper left",
    handlelength=1,
)

add_subplotlabel(axs, location="title")

for ax in axs[-1, :]:
    ax.set_xlabel(label_from_attrs(data["radius_bins"]))

# fig.supxlabel(label_from_attrs(data["radius_bins"]))
fig.tight_layout()

fig.savefig(
    fig_dir / "comparison" / f"condensation-comparison-selected-gridboxes-evaporation-fraction-zoom.pdf"
)

## All microphysics

In [ ]:
def plot_distributions_at_different_heights(
    comparison_dict, compare_key, selected_gridboxes, keys, ylim_data, ylim_diff
):
    # create two figures with shared x and y axis
    # use gridspec, to make len(keys) cols and len(selected_gridboxes) rows
    # add a large axes around each row to add some ylabels
    num_rows = len(selected_gridboxes)
    num_cols = len(keys)

    fig1, axs1 = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(12, 6), sharex=True, sharey=True)
    fig2, axs2 = plt.subplots(
        nrows=num_rows, ncols=num_cols - 1, figsize=(9, 6), sharex=True, sharey=True
    )

    for idx_mp, mp in enumerate(keys):
        cloud_base_idx = 0
        cloud_base_bridbox = selected_gridboxes[cloud_base_idx]

        for idx_height, sel_gridbox in enumerate(selected_gridboxes):
            data = comparison_dict[mp]
            cloud_base_data = data.sel(gridbox=cloud_base_bridbox).compute()
            data = data.sel(gridbox=sel_gridbox).compute()
            # title = textwrap.fill(f'{data_dict[mp]["microphysics"]} ({data_dict[mp]["microphysics_abbrev"]})', 25)
            title = f'{data_dict[mp]["microphysics_abbrev"]}\n'

            plot_mean_median_iqr(
                ax=axs1[idx_height, idx_mp], data=data, dim="cloud_id", x_dim="radius_bins"
            )

            if idx_height > 0:
                axs1[idx_height, idx_mp].plot(
                    cloud_base_data["radius_bins"],
                    cloud_base_data.median("cloud_id"),
                    color="black",
                    linestyle=":",
                    # marker="x", linestyle="None",
                    alpha=1,
                    linewidth=1,
                    zorder=2,
                    label="Cloud\nbase",
                )

            if idx_mp > 0:
                diff = data - comparison_dict[compare_key].sel(gridbox=sel_gridbox).compute()
                diff.attrs["units"] = data.attrs["units"]
                diff.attrs["long_name"] = data.attrs["long_name"]
                plot_mean_median_iqr(
                    ax=axs2[idx_height, idx_mp - 1], data=diff, dim="cloud_id", x_dim="radius_bins"
                )

                axs2[0, idx_mp - 1].set_title(title)
            axs1[0, idx_mp].set_title(title)

    for ax in axs2[:, 1:].flatten():
        ax.axhline(0, color="grey", linestyle="--", alpha=0.5)

    axs1[-1, 0].legend(
        loc="upper left",
        handlelength=1,
    )
    axs2[-1, 0].legend(
        loc="upper left",
        handlelength=1,
    )

    for idx, label in enumerate(labels):
        axs1[idx, 0].set_ylabel(label)
        axs2[idx, 0].set_ylabel(label)

    for axs in [axs1, axs2]:
        ax = axs.flatten()[0]
        ax.set_xscale("log")
        ax.set_yscale("linear")

    axs1[0, 0].set_ylim(ylim_data)
    add_subplotlabel(axs1, location="title")
    fig1.supylabel(label_from_attrs(data, units_appendix=" (\\log\\mu m)^{-1}"))
    # fig1.supxlabel(label_from_attrs(data["radius_bins"]))
    for ax in axs1[-1, :]:
        ax.set_xlabel(label_from_attrs(data["radius_bins"]))
    fig1.tight_layout()

    axs2[0, 0].set_ylim(ylim_diff)
    add_subplotlabel(axs2, location="title")
    fig2.supylabel(label_from_attrs(data, units_appendix=" (\\log\\mu m)^{-1}"))
    # fig1.supxlabel(label_from_attrs(data["radius_bins"]))
    for ax in axs2[-1, :]:
        ax.set_xlabel(label_from_attrs(data["radius_bins"]))
    fig2.tight_layout()
    # fig2.suptitle(f"Anomalies of {label_from_attrs(diff)} compared to condensation only run")

    for fig in [fig1, fig2]:
        fig.tight_layout()

    fig1.savefig(
        fig_dir
        / "comparison"
        / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_normal.pdf"
    )
    fig2.savefig(
        fig_dir
        / "comparison"
        / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.pdf"
    )

    axs1[0, 0].set_xlim(10, None)
    axs2[0, 0].set_xlim(10, None)
    for fig in [fig1, fig2]:
        fig.tight_layout()

    fig1.savefig(
        fig_dir
        / "comparison"
        / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_normal_zoom.pdf"
    )
    fig2.savefig(
        fig_dir
        / "comparison"
        / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.pdf"
    )
    return fig1, fig2

#### Latent cooling

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]

keys = [data.attrs["microphysics_short"] for data in datas]


comparison_dict = latent_heat_comparison_all
compare_key = "condensation"

fig1, fig2 = plot_distributions_at_different_heights(
    comparison_dict=comparison_dict,
    compare_key=compare_key,
    selected_gridboxes=selected_gridboxes,
    keys=keys,  #
    ylim_data=(-1, 20),
    ylim_diff=(-2.5, 2.5),
)

#### Real droplet number concentration

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]

keys = [data.attrs["microphysics_short"] for data in datas]

comparison_dict = multiplicity_comparison_all
compare_key = "condensation"

fig1, fig2 = plot_distributions_at_different_heights(
    comparison_dict=comparison_dict,
    compare_key=compare_key,
    selected_gridboxes=selected_gridboxes,
    keys=keys,  #
    ylim_data=(0, 80),
    ylim_diff=(-8, 8),
)

### Mass

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]

keys = [data.attrs["microphysics_short"] for data in datas]


comparison_dict = mass_comparison_all
compare_key = "condensation"
fig1, fig2 = plot_distributions_at_different_heights(
    comparison_dict=comparison_dict,
    compare_key=compare_key,
    selected_gridboxes=selected_gridboxes,
    keys=keys,  #
    ylim_data=(0, 50),
    ylim_diff=(-2.5, 2.5),
)

### Number concentration

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]

keys = [data.attrs["microphysics_short"] for data in datas]


comparison_dict = number_superdroplets_comparison_all
compare_key = "condensation"

fig1, fig2 = plot_distributions_at_different_heights(
    comparison_dict=comparison_dict,
    compare_key=compare_key,
    selected_gridboxes=selected_gridboxes,
    keys=keys,  #
    ylim_data=(-1, 15),
    ylim_diff=(-1, 3),
)

### correlation histograms


In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 5))
for key in mass_comparison_all:
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    ax.hist(
        xr.corr(
            latent_heat_comparison_all[key],
            mass_comparison_all[key],
            dim=["radius_bins"],
        ).values.flatten(),
        bins=bins,
        color=color,
        alpha=0.5,
        facecolor=color,
    )

condensation
#CA8727


collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 5))
for key in mass_comparison_all:
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    ax.hist(
        xr.corr(
            latent_heat_comparison_all[key],
            number_superdroplets_comparison_all[key],
            dim=["radius_bins"],
        ).values.flatten(),
        bins=bins,
        color=color,
        alpha=0.5,
        facecolor=color,
    )

condensation
#CA8727
collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99


coalbure_condensation_small
#AA4499


In [ ]:
corr_dict = dict()

for mp in data_dict:
    print(mp)
    data = data_dict[mp]["dataset"].sel(time=time_slice).sel(cloud_id=cloud_id_selection)
    da_x = 1e3 * data["latent_cooling_radius_bins"].mean("time", keep_attrs=True)
    da_x = da_x.where(data["sub_cloud_layer_mask"])
    da_x.attrs["units"] = "$mW m^{-3}$"
    da_x.attrs["long_name"] = "Latent cooling"

    da_y = 1e3 * data["mass_represented_per_volume"].mean("time", keep_attrs=True)
    da_y = da_y.where(data["sub_cloud_layer_mask"])
    da_y.attrs["units"] = "$mg m^{-3}$"
    da_y.attrs["long_name"] = "Mass"

    corr_dict[mp] = xr.corr(
        da_x,
        da_y,
        dim=["radius_bins"],
    ).compute()

null_microphysics
condensation
collision_condensation
coalbure_condensation_cke
coalbure_condensation_large
coalbure_condensation_small


In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 5))

for key in corr_dict:
    print(key)
    color = colors_dict[key]
    m_color = adjust_lightness_array([color], 0.9)[0]
    color = adjust_lightness_array([color], 1.3)[0]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    sc = ax.plot(
        corr_dict[key].T,
        data["gridbox_coord3"].T,
        color=color,
        alpha=0.1,
        zorder=0,
    )

    m, s = mean_and_stderror_of_mean(data=corr_dict[key], dims=("cloud_id",))

    ax.plot(
        m,
        data["gridbox_coord3"].mean("cloud_id"),
        color=m_color,
        label=key,
        linewidth=2,
        zorder=2,
    )

    ax.fill_betweenx(
        data["gridbox_coord3"].mean("cloud_id"),
        m - s,
        m + s,
        color=m_color,
        alpha=0.2,
        zorder=1,
    )
ax.set_xlabel("Correlation")
ax.set_ylabel("Height [m]")

null_microphysics
#a6a6a6
condensation
#e3abb4
collision_condensation
#a7c3d1
coalbure_condensation_cke
#dfa85a
coalbure_condensation_large
#70c6b7
coalbure_condensation_small
#c670b7


Text(0, 0.5, 'Height [m]')

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 5))
for key in mass_comparison_all:
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)
    res = xr.corr(
        latent_heat_comparison_all[key],
        mass_comparison_all[key],
        dim=["radius_bins"],
    )

    ax.plot(res.T, res["gridbox"].T, color=color, alpha=0.01)
    ax.plot(res.mean("cloud_id"), res["gridbox"], color=color, linestyle="-", alpha=1);

condensation
#CC6677
collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


In [ ]:
keys = mass_comparison_all.keys()

fig, axs = plt.subplots(ncols=len(keys), figsize=(12, 3))
for i, key in enumerate(keys):
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    axs[i].hist(
        xr.corr(
            latent_heat_comparison_all[key],
            mass_comparison_all[key],
            dim=["radius_bins"],
        ).values.flatten(),
        bins=bins,
        color=color,
        alpha=0.5,
        facecolor=color,
    )

condensation
#CC6677
collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


In [ ]:
keys = mass_comparison_all.keys()

fig, axs = plt.subplots(ncols=len(keys), figsize=(12, 3))
for i, key in enumerate(keys):
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    axs[i].hist(
        xr.corr(
            latent_heat_comparison_all[key],
            multiplicity_comparison_all[key],
            dim=["radius_bins"],
        ).values.flatten(),
        bins=bins,
        color=color,
        alpha=0.5,
        facecolor=color,
    )

condensation
#CC6677
collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


# Temporal evolution of the distributions

In [ ]:
def temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    ds = data_dict[key]["dataset"].sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))
    fig, axs = plt.subplots(
        ncols=2, nrows=1, figsize=(15, 7), sharex=False, sharey=True, width_ratios=[1, 0.5]
    )

    clabel = label_from_attrs(ds[variable])

    pcm = axs[0].pcolormesh(
        ds[variable]["time"],
        ds["gridbox_coord3"],
        ds[variable].sum("radius_bins").compute().T,
        shading="auto",
        cmap=my_cmap,
        norm=my_norm,
    )

    fig.colorbar(mappable=pcm, ax=axs[0], label=clabel, extend="both")
    axs[0].set_xlabel("Time $s$")
    axs[0].set_ylabel("Height $m$")

    ds_stationary = ds.sel(time=slice(400, None))

    axs[1].plot(
        ds_stationary[variable].sum("radius_bins").quantile(0.5, "time").compute(),
        ds_stationary["gridbox_coord3"],
    )
    axs[1].plot(
        ds_stationary[variable].sum("radius_bins").T,
        ds_stationary["gridbox_coord3"],
        color=[0.7, 0.7, 0.7],
        linewidth=0.1,
        alpha=0.1,
        zorder=0,
    )
    axs[1].fill_betweenx(
        ds_stationary["gridbox_coord3"],
        ds_stationary[variable].sum("radius_bins").quantile(0.25, "time").compute(),
        ds_stationary[variable].sum("radius_bins").quantile(0.75, "time").compute(),
        color="k",
        alpha=0.1,
    )
    axs[1].set_xlim(vmin, vmax)
    axs[1].set_xlabel(clabel)

    axs[1].set_ylabel("Height $m$")

    fig.suptitle(
        f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {label_from_attrs(ds[variable], return_units = False)}"
    )

    return fig, axs


def temporal_evolution_different_gridboxes(
    data_dict, key, variable, cloud_id, vmin, vmax, isel_gridboxes=[-1, -2, -3, -5, 10, 0]
):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    ds = data_dict[key]["dataset"].sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))

    clabel = label_from_attrs(ds[variable])

    gridboxes = ds["gridbox"].isel(gridbox=isel_gridboxes).compute()

    width_ratios = np.ones(len(gridboxes) + 1)
    width_ratios[-1] = 0.1

    fig, axs = plt.subplots(
        ncols=len(gridboxes) + 1,
        nrows=1,
        figsize=(15, 10),
        sharex=False,
        sharey=False,
        width_ratios=width_ratios,
    )

    for idx, gridbox in enumerate(gridboxes):
        altitude = ds["gridbox_coord3"].sel(gridbox=gridbox).compute()
        ds_selected = ds.sel(gridbox=gridbox)
        pcm = axs[idx].pcolormesh(
            ds_selected[variable]["radius_bins"],
            ds_selected[variable]["time"],
            ds_selected[variable].compute(),
            shading="auto",
            cmap=my_cmap,
            norm=my_norm,
        )
        axs[idx].set_title(f"Gridbox {gridbox.data}\n{altitude.data}m")
    for ax in axs[:-1]:
        ax.set_xscale("log")
        ax.set_xticks([1e0, 1e1, 1e2, 1e3])
        ax.set_xlim(1e-1, 1e3)
        ax.grid()
        ax.set_xlabel("Radius $\\mu m$")

    for ax in axs[1:-1]:
        ax.set_yticklabels([])

    axs[0].set_ylabel("Time $s$")

    fig.colorbar(
        cax=axs[-1],
        mappable=pcm,
        orientation="vertical",
        label=clabel,
        extend="both",
    )
    fig.suptitle(
        f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {label_from_attrs(ds[variable], return_units = False)}"
    )

    return fig, axs

#### Multiplicity

In [ ]:
vmin, vmax = 1e-10, 1000
vmin_small, vmax_small = vmin, 50
cloud_id = 142
variable = "xi_per_volume"

keys = [
    "null_microphysics",
    "condensation",
    "collision_condensation",
    # "coalbure_condensation_cke",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]
isel_gridboxes = [-1, -2, 15, 1]
for key in keys:
    # fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.png", dpi=600)
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.pdf")
    fig, axs = temporal_evolution_different_gridboxes(
        data_dict=data_dict,
        key=key,
        variable=variable,
        cloud_id=cloud_id,
        vmin=vmin_small,
        vmax=vmax_small,
        isel_gridboxes=isel_gridboxes,
    )
    fig.savefig(
        fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.png",
        dpi=200,
    )
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.pdf")

### Mass

In [ ]:
vmin, vmax = 1e-10, 0.2
vmin_small, vmax_small = vmin, 0.015
variable = "mass_represented_per_volume"
cloud_id = 142

keys = [
    "null_microphysics",
    "condensation",
    "collision_condensation",
    # "coalbure_condensation_cke",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

isel_gridboxes = [-1, -2, 15, 1]
for key in keys:
    # fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.png", dpi=600)
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.pdf")
    fig, axs = temporal_evolution_different_gridboxes(
        data_dict=data_dict,
        key=key,
        variable=variable,
        cloud_id=cloud_id,
        vmin=vmin_small,
        vmax=vmax_small,
        isel_gridboxes=isel_gridboxes,
    )
    fig.savefig(
        fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.png",
        dpi=200,
    )
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.pdf")

### Number of SDs

In [ ]:
vmin, vmax = 1e-10, 40
vmin_small, vmax_small = vmin, 2
variable = "number_superdroplets_per_volume"
cloud_id = 142

keys = [
    "null_microphysics",
    "condensation",
    "collision_condensation",
    # "coalbure_condensation_cke",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

isel_gridboxes = [-1, -2, 15, 1]
for key in keys:
    # fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.png", dpi=600)
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.pdf")
    fig, axs = temporal_evolution_different_gridboxes(
        data_dict=data_dict,
        key=key,
        variable=variable,
        cloud_id=cloud_id,
        vmin=vmin_small,
        vmax=vmax_small,
        isel_gridboxes=isel_gridboxes,
    )
    fig.savefig(
        fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.png",
        dpi=200,
    )
    # fig.savefig(bbox_inches='tight', fname= fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.pdf")

### Latent cooling

In [ ]:
vmin, vmax = -0.02, -1e-10
variable = "latent_cooling_radius_bins"
cloud_id = 222
key = "condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax)

In [ ]:
vmin, vmax = -0.02, -1e-10
variable = "latent_cooling_radius_bins"
cloud_id = 222
key = "collision_condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax)